# The goal of this notebook: To see how many public schools are within a mile of superfund sites. 

## Let's load the clean superfund dataset. 

In [1]:
from pathlib import Path
import pandas as pd

ROOT = Path('..') 
DATA = ROOT / 'all_datasets' / 'Superfund_list_csv'

df = pd.read_csv(DATA / 'superfund.csv')
df

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df['Status'].value_counts()

Status
NPL Site             1340
Deleted NPL Site      458
Proposed NPL Site      42
Name: count, dtype: int64

## We are only interested in active superfund sites. 
## We are not interested in Superfund/NPL site that no longer exists or proposed ones. Let's drop those. 

In [ ]:
df = df[~df['Status'].isin([
    'Deleted NPL Site',
    'Proposed NPL Site'
])]
df

,X,Y,OBJECTID,Site_Name,Site_Score,Site_EPA_ID,SEMS_ID,SITS_ID,Region_ID,State,...,Deletion_FR_Notice,Final_FR_Notice,NOID_FR_Notice,Restoration_FR_Notice_Jumper_Pa,Site_has_had_a_Partial_Deletion,CreationDate,Creator,EditDate,Editor,ObjectId2
0,-8.174407e+06,5.031617e+06,NaN,Kellogg-Deering Well Field,39.92,CTD980670814,100252,5,1,Connecticut,...,NaN,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,1
4,-8.090855e+06,5.083566e+06,NaN,Durham Meadows,33.94,CTD001452093,100108,9,1,Connecticut,...,NaN,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,5
6,-8.011323e+06,5.152080e+06,NaN,Linemaster Switch Corp.,33.71,CTD001153923,100041,1095,1,Connecticut,...,NaN,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,7
7,-8.131580e+06,5.158067e+06,NaN,Barkhamsted-New Hartford Landfill,38.05,CTD980732333,100255,1206,1,Connecticut,...,NaN,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,8
8,-8.006561e+06,5.110971e+06,NaN,Gallup's Quarry,46.29,CTD108960972,100201,1209,1,Connecticut,...,NaN,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,-8.388336e+06,4.892236e+06,NaN,North Penn - Area 12,28.90,PAD057152365,301012,1057,3,Pennsylvania,...,NaN,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,1836
1836,-8.380008e+06,4.897681e+06,NaN,North Penn - Area 7,35.57,PAD002498632,300629,1058,3,Pennsylvania,...,NaN,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,1837
1837,-8.381872e+06,4.902095e+06,NaN,North Penn - Area 6,35.57,PAD980926976,301733,1059,3,Pennsylvania,...,NaN,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,"<a href=""https://www.epa.gov/superfund/partial...",NaN,NaN,NaN,NaN,1838
1838,-8.382111e+06,4.908742e+06,NaN,North Penn - Area 2,35.57,PAD002342475,300579,1060,3,Pennsylvania,...,NaN,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,1839


## Time to convert df to geo dfs EPSG:5070 – NAD83 
### I am doing this to do buffering and later see how close public schools are to the superfund sites.

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

/var/folders/1_/vh5ppk9j3vvc6l2z4r6n7dbw0000gn/T/ipykernel_905/3774028759.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)


## Convert DataFrame to GeoDataFrame by assigning the geometry column
## and specifying that coordinates are in latitude/longitude (EPSG:4326)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf

,X,Y,OBJECTID,Site_Name,Site_Score,Site_EPA_ID,SEMS_ID,SITS_ID,Region_ID,State,...,Final_FR_Notice,NOID_FR_Notice,Restoration_FR_Notice_Jumper_Pa,Site_has_had_a_Partial_Deletion,CreationDate,Creator,EditDate,Editor,ObjectId2,geometry
0,-8.174407e+06,5.031617e+06,NaN,Kellogg-Deering Well Field,39.92,CTD980670814,100252,5,1,Connecticut,...,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,1,POINT (-73.43195 41.13055)
4,-8.090855e+06,5.083566e+06,NaN,Durham Meadows,33.94,CTD001452093,100108,9,1,Connecticut,...,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,5,POINT (-72.68139 41.48111)
6,-8.011323e+06,5.152080e+06,NaN,Linemaster Switch Corp.,33.71,CTD001153923,100041,1095,1,Connecticut,...,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,7,POINT (-71.96694 41.94056)
7,-8.131580e+06,5.158067e+06,NaN,Barkhamsted-New Hartford Landfill,38.05,CTD980732333,100255,1206,1,Connecticut,...,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,8,POINT (-73.04722 41.98056)
8,-8.006561e+06,5.110971e+06,NaN,Gallup's Quarry,46.29,CTD108960972,100201,1209,1,Connecticut,...,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,9,POINT (-71.92416 41.66528)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,-8.388336e+06,4.892236e+06,NaN,North Penn - Area 12,28.90,PAD057152365,301012,1057,3,Pennsylvania,...,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,1836,POINT (-75.3537 40.1807)
1836,-8.380008e+06,4.897681e+06,NaN,North Penn - Area 7,35.57,PAD002498632,300629,1058,3,Pennsylvania,...,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,1837,POINT (-75.27889 40.21806)
1837,-8.381872e+06,4.902095e+06,NaN,North Penn - Area 6,35.57,PAD980926976,301733,1059,3,Pennsylvania,...,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,"<a href=""https://www.epa.gov/superfund/partial...",NaN,NaN,NaN,NaN,1838,POINT (-75.29564 40.24833)
1838,-8.382111e+06,4.908742e+06,NaN,North Penn - Area 2,35.57,PAD002342475,300579,1060,3,Pennsylvania,...,"<a href=""https://semspub.epa.gov/src/document/...",NaN,NaN,No,NaN,NaN,NaN,NaN,1839,POINT (-75.29778 40.29389)


In [ ]:
gdf = gdf.to_crs(epsg=5070)
gdf['original_point'] = gdf.geometry.copy()

###  Reproject to NAD83 / CONUS Albers (EPSG:5070) for accurate distance buffering
### I will set it to one mile radius! 
### Conversion factor: 1 foot = 0.3048 meters, 1 mile = 5280 feet

In [ ]:
mile_in_meters = 5280 * 0.3048
gdf['buffered'] = gdf.geometry.buffer(mile_in_meters)

## Now I want to get out my private school and public school data. 

### Here is where I got my public school dataset from: https://nces.ed.gov/programs/edge/geographic/schoollocations

In [ ]:
ROOT = Path('..') 
DATA = ROOT / 'all_datasets' / 'public_school_dataset'

df_1 = pd.read_excel (DATA / 'publicschools.xlsx')
df_1

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,CD,SLDL,SLDU,SCHOOLYEAR
0,10000500870,100005,Albertville Middle School,1,600 E Alabama Ave,Albertville,AL,35950,1,1095,...,-86.206200,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
1,10000500871,100005,Albertville High School,1,402 E McCord Ave,Albertville,AL,35950,1,1095,...,-86.204900,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
2,10000500879,100005,Albertville Intermediate School,1,901 W McKinney Ave,Albertville,AL,35950,1,1095,...,-86.220100,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
3,10000500889,100005,Albertville Elementary School,1,145 West End Drive,Albertville,AL,35950,1,1095,...,-86.221806,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
4,10000501616,100005,Albertville Kindergarten and PreK,1,257 Country Club Rd,Albertville,AL,35951,1,1095,...,-86.193300,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102269,780003000024,7800030,Lockhart Elementary School,78,41 ESTATE THOMAS,Saint Thomas,VI,802,78,78030,...,-64.917602,N,N,0,N,N,7898,N,N,2023-2024
102270,780003000026,7800030,Ulla F. Muller Elementary School,78,7B ESTATE CONTANT,Saint Thomas,VI,802,78,78030,...,-64.952483,N,N,0,N,N,7898,N,N,2023-2024
102271,780003000027,7800030,Yvonne E. Milliner-Bowsky Elementary School,78,15B and 16 ESTATE MANDAHL,Saint Thomas,VI,802,78,78030,...,-64.899024,N,N,0,N,N,7898,N,N,2023-2024
102272,780003000033,7800030,Addelita Cancryn School,78,1 CROWN BAY,Saint Thomas,VI,802,78,78030,...,-64.945940,N,N,0,N,N,7898,N,N,2023-2024


## Yay! Now time to convert this public school datasetto geo dfs EPSG:5070 – NAD83. 

In [ ]:
geometry = [Point(xy) for xy in zip(df_1['LON'], df_1['LAT'])]

gdf_school = gpd.GeoDataFrame(
    df_1,
    geometry=geometry,
    crs="EPSG:4326"
)

gdf_school = gdf_school.to_crs(epsg=5070)
gdf_school

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,CD,SLDL,SLDU,SCHOOLYEAR,geometry
0,10000500870,100005,Albertville Middle School,1,600 E Alabama Ave,Albertville,AL,35950,1,1095,...,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024,POINT (893450.353 1290139.768)
1,10000500871,100005,Albertville High School,1,402 E McCord Ave,Albertville,AL,35950,1,1095,...,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024,POINT (893545.523 1290374.393)
2,10000500879,100005,Albertville Intermediate School,1,901 W McKinney Ave,Albertville,AL,35950,1,1095,...,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024,POINT (892036.359 1291465.947)
3,10000500889,100005,Albertville Elementary School,1,145 West End Drive,Albertville,AL,35950,1,1095,...,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024,POINT (892117.83 1289159.146)
4,10000501616,100005,Albertville Kindergarten and PreK,1,257 Country Club Rd,Albertville,AL,35951,1,1095,...,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024,POINT (894282.129 1293552.691)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102269,780003000024,7800030,Lockhart Elementary School,78,41 ESTATE THOMAS,Saint Thomas,VI,802,78,78030,...,N,N,0,N,N,7898,N,N,2023-2024,POINT (3350974.854 51429.571)
102270,780003000026,7800030,Ulla F. Muller Elementary School,78,7B ESTATE CONTANT,Saint Thomas,VI,802,78,78030,...,N,N,0,N,N,7898,N,N,2023-2024,POINT (3347458.925 49876.432)
102271,780003000027,7800030,Yvonne E. Milliner-Bowsky Elementary School,78,15B and 16 ESTATE MANDAHL,Saint Thomas,VI,802,78,78030,...,N,N,0,N,N,7898,N,N,2023-2024,POINT (3352466.041 53380.086)
102272,780003000033,7800030,Addelita Cancryn School,78,1 CROWN BAY,Saint Thomas,VI,802,78,78030,...,N,N,0,N,N,7898,N,N,2023-2024,POINT (3348210.389 49896.546)


## Spatially join public schools to buffered Superfund sites
### to identify schools located within the buffer distance

In [ ]:
gdf_buffer = gdf[['buffered', 'original_point']].copy()
gdf_buffer = gdf_buffer.rename(columns={'buffered': 'geometry'}).set_geometry('geometry')

In [ ]:
joined = gpd.sjoin(gdf_school, gdf_buffer, how='inner', predicate='intersects')
joined

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,CBSATYPE,CSA,NMCSA,CD,SLDL,SLDU,SCHOOLYEAR,geometry,index_right,original_point
1581,20001000080,200010,Adak School,2,100 Mechanics Rd,Adak,AK,99546,2,2016,...,0,N,N,200,02037,0200S,2023-2024,POINT (-5048042.609 5481350.169),406,POINT (-5047853.689 5482664.618)
1692,20018000118,200180,Ursa Minor Elementary,2,336 Hoonah Ave,JBER,AK,99505,2,2020,...,1,N,N,200,02018,0200I,2023-2024,POINT (-3078777.726 5071617.835),407,POINT (-3078812.125 5072987.861)
1709,20018000400,200180,Ursa Major Elementary,2,454 Dyea St,JBER,AK,99505,2,2020,...,1,N,N,200,02018,0200I,2023-2024,POINT (-3078039.909 5071677.495),407,POINT (-3078812.125 5072987.861)
1907,20060000238,200600,Anderson Crawford Elementary,2,692 Raven's Way,Eielson AFB,AK,99702,2,2090,...,1,N,N,200,02034,0200Q,2023-2024,POINT (-2775253.833 5271868.163),404,POINT (-2774173.406 5271097.718)
1908,20060000256,200600,Anderson Elementary,2,768 Kodiak St,Eielson AFB,AK,99702,2,2090,...,1,N,N,200,02034,0200Q,2023-2024,POINT (-2774949.653 5272493.475),404,POINT (-2774173.406 5271097.718)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102213,720003002019,7200030,INES MARIA MENDOZA DE MUNOZ MARIN CABO ROJO,72,CALLE BARBOSA NORTE URB ANA MARIA,CABO ROJO,PR,623,72,72023,...,1,364,"Mayagüez-Aguadilla, PR",7298,72020,72004,2023-2024,POINT (3126324.894 -50074.522),1269,POINT (3126218.406 -50039.378)
102217,720003002026,7200030,AGAPITO ROSARIO ROSARIO,72,CALLE S ESQ T URB ALTURAS,VEGA BAJA,PR,693,72,72145,...,1,490,"San Juan-Bayamón, PR",7298,72012,72003,2023-2024,POINT (3193270.65 9667.391),1260,POINT (3194551.966 10080.644)
102243,720003002079,7200030,SEVERO E. COLBERG RAMIREZ,72,102 AVE PEDRO ALBIZU CAMPOS,CABO ROJO,PR,623,72,72023,...,1,364,"Mayagüez-Aguadilla, PR",7298,72020,72004,2023-2024,POINT (3125931.186 -50536.028),1269,POINT (3126218.406 -50039.378)
102245,720003002085,7200030,JUAN QUIRINDONGO MORELL,72,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,693,72,72145,...,1,490,"San Juan-Bayamón, PR",7298,72012,72003,2023-2024,POINT (3193610.024 9507.784),1260,POINT (3194551.966 10080.644)


In [ ]:
joined['distance_to_site_m'] = joined.apply(
    lambda row: row.geometry.distance(gdf_buffer.loc[row.index_right, 'original_point']),
    axis=1
)
joined['distance_to_site_miles'] = joined['distance_to_site_m'] / 1609.34
joined

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,NMCSA,CD,SLDL,SLDU,SCHOOLYEAR,geometry,index_right,original_point,distance_to_site_m,distance_to_site_miles
1581,20001000080,200010,Adak School,2,100 Mechanics Rd,Adak,AK,99546,2,2016,...,N,200,02037,0200S,2023-2024,POINT (-5048042.609 5481350.169),406,POINT (-5047853.689 5482664.618),1327.955801,0.825156
1692,20018000118,200180,Ursa Minor Elementary,2,336 Hoonah Ave,JBER,AK,99505,2,2020,...,N,200,02018,0200I,2023-2024,POINT (-3078777.726 5071617.835),407,POINT (-3078812.125 5072987.861),1370.457861,0.851565
1709,20018000400,200180,Ursa Major Elementary,2,454 Dyea St,JBER,AK,99505,2,2020,...,N,200,02018,0200I,2023-2024,POINT (-3078039.909 5071677.495),407,POINT (-3078812.125 5072987.861),1520.979033,0.945095
1907,20060000238,200600,Anderson Crawford Elementary,2,692 Raven's Way,Eielson AFB,AK,99702,2,2090,...,N,200,02034,0200Q,2023-2024,POINT (-2775253.833 5271868.163),404,POINT (-2774173.406 5271097.718),1326.992519,0.824557
1908,20060000256,200600,Anderson Elementary,2,768 Kodiak St,Eielson AFB,AK,99702,2,2090,...,N,200,02034,0200Q,2023-2024,POINT (-2774949.653 5272493.475),404,POINT (-2774173.406 5271097.718),1597.090391,0.992388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102213,720003002019,7200030,INES MARIA MENDOZA DE MUNOZ MARIN CABO ROJO,72,CALLE BARBOSA NORTE URB ANA MARIA,CABO ROJO,PR,623,72,72023,...,"Mayagüez-Aguadilla, PR",7298,72020,72004,2023-2024,POINT (3126324.894 -50074.522),1269,POINT (3126218.406 -50039.378),112.137600,0.069679
102217,720003002026,7200030,AGAPITO ROSARIO ROSARIO,72,CALLE S ESQ T URB ALTURAS,VEGA BAJA,PR,693,72,72145,...,"San Juan-Bayamón, PR",7298,72012,72003,2023-2024,POINT (3193270.65 9667.391),1260,POINT (3194551.966 10080.644),1346.309783,0.836560
102243,720003002079,7200030,SEVERO E. COLBERG RAMIREZ,72,102 AVE PEDRO ALBIZU CAMPOS,CABO ROJO,PR,623,72,72023,...,"Mayagüez-Aguadilla, PR",7298,72020,72004,2023-2024,POINT (3125931.186 -50536.028),1269,POINT (3126218.406 -50039.378),573.721492,0.356495
102245,720003002085,7200030,JUAN QUIRINDONGO MORELL,72,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,693,72,72145,...,"San Juan-Bayamón, PR",7298,72012,72003,2023-2024,POINT (3193610.024 9507.784),1260,POINT (3194551.966 10080.644),1102.462918,0.685040


## Add the SITE_ID from gdf to joined using index_right

In [ ]:
joined['Site_Name'] = joined['index_right'].apply(lambda i: gdf.loc[i, 'Site_Name'])


In [ ]:
final_schools = joined[[
    'NAME',                
    'Site_Name',              
    'distance_to_site_miles'
]].sort_values('distance_to_site_miles').reset_index(drop=True)

final_schools

,NAME,Site_Name,distance_to_site_miles
0,LONGFELLOW ELEMENTARY,Fruit Avenue Plume,0.045738
1,Insight School of WA Open Doors Program,"Commencement Bay, South Tacoma Channel",0.050809
2,Basin School,Basin Mining Area,0.059319
3,Van Cleve Elementary School,East Troy Contaminated Aquifer,0.066303
4,North Belmont Elementary,North Belmont PCE,0.068183
...,...,...,...
2055,GREEN PINES ELEM.,Ellisville Site,0.997433
2056,Cordova High School,National Fireworks,0.998665
2057,Owen Valley Middle School,Franklin Street Groundwater Contamination,0.998830
2058,SULLIVAN ELEM.,Oak Grove Village Well,0.998947


## Save this finals_schools as a csv! 

In [ ]:
from pathlib import Path

REPO_ROOT = Path.cwd().parent

output_path = REPO_ROOT / 'all_datasets' / 'schools_and_superfund' / 'schools_and_superfund.csv'


output_path.parent.mkdir(parents=True, exist_ok=True)

final_schools.to_csv(output_path, index=False)

## END. 